In [36]:
import spacy
import re
from nltk.stem import PorterStemmer
stemmer = PorterStemmer() # Inicializa un stemmer de Porter para reducir palabras a su raíz

# Cargar el modelo en español de spaCy
nlp = spacy.load("es_core_news_sm") # Carga un modelo de procesamiento de lenguaje natural en español


# Dada la oración y el aspecto, extrae el segmento donde se encuentra el aspecto
def extraer_segmento(oracion,aspecto):
    """
    Extrae el segmento de una oración que contiene un aspecto dado. Combina varias funciones para identificar y retornar la porción relevante de texto.
    """
    dependencias = extraer_dependencias(oracion, aspecto) # Obtiene las dependencias sintácticas del aspecto en la oración
    lista = palabras_sin_repetir(dependencias) # Elimina las palabras repetidas de la lista de dependencias (función no definida aquí)
    segmento = combinacion(oracion,lista) # Combina las palabras en un segmento coherente (función no definida aquí)

    return convertir_lista_oracion(segmento) # Convierte la lista de palabras resultante en una oración (función no definida aquí)


def extraer_dependencias(texto_opinion, palabra_clave):
    """
    Extrae información sintáctica relacionada con una palabra clave en un texto. Identifica y devuelve frases relacionadas con el aspecto.
    """
    doc_analizado = nlp(texto_opinion) # Analiza el texto usando el modelo de spaCy
    frases_extraidas = [] # Inicializa una lista para guardar las frases extraídas

    for palabra_actual in doc_analizado: # Itera sobre cada palabra del texto analizado
        if palabra_actual.text == palabra_clave or palabra_actual.text == stemmer.stem(palabra_clave) : # Busca la palabra clave o su raíz

            palabras_asociadas = [] # Lista para guardar las palabras asociadas a la palabra clave

            # Extrae palabras a la izquierda y derecha de la palabra clave
            palabras_asociadas.extend(extraer_palabras_adyacentes(palabra_actual, 'left'))
            palabras_asociadas.append(palabra_actual.text)
            palabras_asociadas.extend(extraer_palabras_adyacentes(palabra_actual, 'right'))

            palabra_principal = palabra_actual.head # Encuentra la palabra principal (la raíz sintáctica) de la palabra clave

            # Extrae sujetos y objetos directos relacionados con la palabra principal
            palabras_asociadas.extend(extraer_palabras_adyacentes(palabra_principal, 'left'))
            palabras_asociadas.append(palabra_principal.text)
            palabras_asociadas.extend(extraer_palabras_adyacentes(palabra_principal, 'right'))

            # Agrega n-gramas, contexto y la frase extraída a la lista de frases
            frases_extraidas.append(palabras_asociadas)
            frases_extraidas.append(n_gramas(texto_opinion, palabra_clave, 3))
            frases_extraidas.append(extraccion_contexto(texto_opinion, palabra_clave))

    return frases_extraidas # Devuelve la lista de frases extraídas


# Funciones auxiliares para extraer dependencias

def extraer_palabras_adyacentes(token_actual, direccion):
    """
    Extrae las palabras a la izquierda o derecha de un token, incluyendo adjetivos relacionados con sustantivos.

    Args:
        token_actual: El token de referencia.
        direccion: 'left' para palabras a la izquierda, 'right' para palabras a la derecha.

    Returns:
        Una lista de palabras.
    """
    palabras = []
    adyacentes = token_actual.lefts if direccion == 'left' else token_actual.rights
    for token_adyacente in adyacentes:
        palabras.append(token_adyacente.text)
        if token_adyacente.dep_ == 'NOUN':
            for token_adjetivo in token_adyacente.lefts:
                if token_adjetivo.dep_ == 'ADJ':
                    palabras.append(token_adjetivo.text)
            for token_adjetivo in token_adyacente.rights:
                if token_adjetivo.dep_ == 'ADJ':
                    palabras.append(token_adjetivo.text)
    return palabras

def n_gramas(oracion, palabra, numero):
    """
    Extrae n-gramas (secuencias de n palabras) alrededor de una palabra clave en una oración.
    """
    palabras = limpiar_oracion(oracion).split() # Limpia la oración y la divide en palabras 
    index_palabra = [] # Inicializa una lista para guardar el índice de la palabra clave en la lista de palabras

    for index, word in enumerate(palabras): # Itera sobre las palabras para encontrar la palabra clave
        if word.lower() == palabra.lower() or word.lower() == stemmer.stem(palabra.lower()): # Busca la palabra clave o su raíz
            index_palabra = index
            break

    palabras_anteriores = palabras[max(0, index_palabra - numero):index_palabra] # Extrae las palabras anteriores a la palabra clave
    palabras_posteriores = palabras[index_palabra + 1:index_palabra + 1 + numero] # Extrae las palabras posteriores a la palabra clave

    return palabras_anteriores + [palabra] + palabras_posteriores # Devuelve una lista con las palabras anteriores, la palabra clave y las palabras posteriores


def extraccion_contexto(oracion, palabra_relevante):
    """
    Extrae el contexto de una palabra clave en una oración, incluyendo modificadores.
    """
    doc = nlp(oracion) # Analiza la oración con spaCy
    token_palabra_relevante = None # Inicializa la variable para guardar el token de la palabra clave
    for token in doc: # Busca el token de la palabra clave en la oración analizada
        if token.text.lower() == palabra_relevante:
            token_palabra_relevante = token
            break

    if not token_palabra_relevante: # Si no se encuentra la palabra clave
        return "Palabra relevante no encontrada en la oración"

    modificadores = [] # Inicializa una lista para guardar los modificadores de la palabra clave
    for token in doc: # Itera sobre los tokens de la oración analizada
        # Busca modificadores de la palabra clave (adjetivos, adverbios, etc.)
        if token.head == token_palabra_relevante and token.dep_ in ["amod", "advmod", "nsubj", "nummod", "det"]: 
            modificadores.append(token.text)

    return modificadores # Devuelve la lista de modificadores


def limpiar_oracion(oracion):

    # Usar una expresión regular para encontrar y reemplazar todos los caracteres que no sean letras
    oracion_limpia = re.sub(r'[^a-zA-Z\sáéíóúüÁÉÍÓÚÜñÑ]', ' ', oracion) 
    # Devolver la oración limpia
    return oracion_limpia

def palabras_sin_repetir(lista_de_listas):
    palabras = set()  # Utilizamos un conjunto para evitar palabras duplicadas  
    for sublista in lista_de_listas:
        for palabra in sublista:
            palabras.add(palabra) 

    return list(palabras)  # Convertimos el conjunto de palabras de nuevo a una lista y la devolvemos

def combinacion(oracion, lista_palabras):
    """
    Extrae las palabras de una oración que están presentes en una lista dada, 
    evitando duplicados y manejando algunas variaciones simples.

    Args:
        oracion: La oración de entrada como una cadena de texto.
        lista_palabras: Una lista de palabras a buscar en la oración.

    Returns:
        Una lista de palabras de la oración que se encontraron en la lista de palabras, sin duplicados.
    """
    resultado = [] # Inicializa una lista vacía para almacenar las palabras encontradas.

    for token in oracion.split(): # Itera sobre cada palabra (token) en la oración.
        if (token in lista_palabras) or (token[0:-1] in lista_palabras): # Verifica si el token o su forma sin el último carácter está en la lista de palabras.
            if not token in resultado: # Verifica si el token ya está en la lista de resultados para evitar duplicados.
                resultado.append(token) # Agrega el token a la lista de resultados.

    return resultado # Devuelve la lista de palabras encontradas.

def convertir_lista_oracion(lista_palabras):

    oracion = " ".join(lista_palabras)  # Unir las palabras con espacios
    return oracion






import csv

def process_dataset(input_filename, output_filename):
    """
    Procesa un dataset y crea un nuevo CSV con las columnas 'texto', 'aspecto' y 'polaridad'.
    """
    with open(input_filename, 'r', encoding='utf-8') as infile, open(output_filename, 'w', newline='', encoding='utf-8') as outfile:
        reader = csv.reader(infile, delimiter=';')
        writer = csv.writer(outfile, delimiter=',')
        writer.writerow(['texto', 'aspecto', 'polaridad']) #Escribe la cabecera

        next(reader) # Salta la cabecera del archivo de entrada

        for row in reader:
            text_tokens = eval(row[0]) #Evalua la cadena para obtener la lista de tokens
            tags = [x for x in row[1].split(',')] # Convierte las tags a enteros
            polarities = [x for x in row[2].split(',')] # Convierte las polaridades a enteros

            sentence = ' '.join(text_tokens) # Junta los tokens para formar la oración


            for i, tag in enumerate(tags):
                if tag[1] == '1': #Si es un aspecto positivo o negativo (0 o 1)
                    aspect = text_tokens[i]
                    polarity = polarities[i][1]
 

                    writer.writerow([sentence, aspect, polarity])

In [37]:
# Ejemplo de uso:
import pandas as pd


input_file = '../Data/dataset_test_without_duplicates.csv'


output_file = 'output.csv'
process_dataset(input_file, output_file)

print(f"Archivo CSV procesado y guardado en: {output_file}")

Archivo CSV procesado y guardado en: output.csv


In [38]:
import xml.etree.ElementTree as ET
import csv

def process_xml_to_csv(xml_filename, csv_filename):
    """
    Procesa un archivo XML de reseñas y crea un archivo CSV.
    """
    tree = ET.parse(xml_filename)
    root = tree.getroot()

    data = []
    for review in root.findall('Review'):
        for sentence in review.findall('sentences/sentence'):
            text = sentence.find('text').text
            for opinion in sentence.findall('Opinions/Opinion'):
                target = opinion.get('target')
                polarity = opinion.get('polarity')
                if target is None:
                    target = "GENERAL" # para opiniones sin un target especifico
                data.append([text, target, 1 if polarity == 'positive' else 0])


    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['texto', 'aspecto', 'polaridad'])
        writer.writerows(data)

#Ejemplo de uso: Reemplaza 'input.xml' y 'output.csv' con los nombres de tus archivos.
xml_file = '../Data/SemEval/SemEval-2016ABSA Restaurants-Spanish_Test.xml'
csv_file = 'output3.csv'
process_xml_to_csv(xml_file, csv_file)
print(f"Archivo CSV creado: {csv_file}")



Archivo CSV creado: output3.csv


In [39]:
import xml.etree.ElementTree as ET
import csv

def process_xml_to_csv(xml_filename, csv_filename):
    """
    Procesa un archivo XML de reseñas y crea un archivo CSV, extrayendo solo la primera palabra del aspecto.
    """
    tree = ET.parse(xml_filename)
    root = tree.getroot()

    data = []
    for review in root.findall('Review'):
        for sentence in review.findall('sentences/sentence'):
            text = sentence.find('text').text
            for opinion in sentence.findall('Opinions/Opinion'):
                target = opinion.get('target')
                polarity = opinion.get('polarity')

                if target is not None:
                    aspect = target.split()[0] # Extrae la primera palabra
                else:
                    aspect = "GENERAL" # Para opiniones sin target específico

                data.append([text, aspect, 1 if polarity == 'positive' else 0])

    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['texto', 'aspecto', 'polaridad'])
        writer.writerows(data)

# Ejemplo de uso: Reemplaza 'input.xml' y 'output.csv' con los nombres de tus archivos.
xml_file = '../Data/SemEval/SemEval-2016ABSA Restaurants-Spanish_Test.xml'
csv_file = 'output3.csv'
process_xml_to_csv(xml_file, csv_file)
print(f"Archivo CSV creado: {csv_file}")



Archivo CSV creado: output3.csv


In [40]:
from os import sep
import spacy
import re
from nltk.stem import PorterStemmer
import xml.etree.ElementTree as ET
import csv
stemmer = PorterStemmer() # Inicializa un stemmer de Porter para reducir palabras a su raíz

# Cargar el modelo en español de spaCy
nlp = spacy.load("es_core_news_sm") # Carga un modelo de procesamiento de lenguaje natural en español


# Dada la oración y el aspecto, extrae el segmento donde se encuentra el aspecto
def extraer_segmento(oracion,aspecto):
    """
    Extrae el segmento de una oración que contiene un aspecto dado. Combina varias funciones para identificar y retornar la porción relevante de texto.
    """
    dependencias = extraer_dependencias(oracion, aspecto) # Obtiene las dependencias sintácticas del aspecto en la oración
    lista = palabras_sin_repetir(dependencias) # Elimina las palabras repetidas de la lista de dependencias (función no definida aquí)
    segmento = combinacion(oracion,lista) # Combina las palabras en un segmento coherente (función no definida aquí)

    return convertir_lista_oracion(segmento) # Convierte la lista de palabras resultante en una oración (función no definida aquí)


def extraer_dependencias(texto_opinion, palabra_clave):
    """
    Extrae información sintáctica relacionada con una palabra clave en un texto. Identifica y devuelve frases relacionadas con el aspecto.
    """
    doc_analizado = nlp(texto_opinion) # Analiza el texto usando el modelo de spaCy
    frases_extraidas = [] # Inicializa una lista para guardar las frases extraídas

    for palabra_actual in doc_analizado: # Itera sobre cada palabra del texto analizado
        if palabra_actual.text == palabra_clave or palabra_actual.text == stemmer.stem(palabra_clave) : # Busca la palabra clave o su raíz

            palabras_asociadas = [] # Lista para guardar las palabras asociadas a la palabra clave

            # Extrae palabras a la izquierda y derecha de la palabra clave
            palabras_asociadas.extend(extraer_palabras_adyacentes(palabra_actual, 'left'))
            palabras_asociadas.append(palabra_actual.text)
            palabras_asociadas.extend(extraer_palabras_adyacentes(palabra_actual, 'right'))

            palabra_principal = palabra_actual.head # Encuentra la palabra principal (la raíz sintáctica) de la palabra clave

            # Extrae sujetos y objetos directos relacionados con la palabra principal
            palabras_asociadas.extend(extraer_palabras_adyacentes(palabra_principal, 'left'))
            palabras_asociadas.append(palabra_principal.text)
            palabras_asociadas.extend(extraer_palabras_adyacentes(palabra_principal, 'right'))

            # Agrega n-gramas, contexto y la frase extraída a la lista de frases
            frases_extraidas.append(palabras_asociadas)
            frases_extraidas.append(n_gramas(texto_opinion, palabra_clave, 3))
            frases_extraidas.append(extraccion_contexto(texto_opinion, palabra_clave))

    return frases_extraidas # Devuelve la lista de frases extraídas


# Funciones auxiliares para extraer dependencias

def extraer_palabras_adyacentes(token_actual, direccion):
    """
    Extrae las palabras a la izquierda o derecha de un token, incluyendo adjetivos relacionados con sustantivos.

    Args:
        token_actual: El token de referencia.
        direccion: 'left' para palabras a la izquierda, 'right' para palabras a la derecha.

    Returns:
        Una lista de palabras.
    """
    palabras = []
    adyacentes = token_actual.lefts if direccion == 'left' else token_actual.rights
    for token_adyacente in adyacentes:
        palabras.append(token_adyacente.text)
        if token_adyacente.dep_ == 'NOUN':
            for token_adjetivo in token_adyacente.lefts:
                if token_adjetivo.dep_ == 'ADJ':
                    palabras.append(token_adjetivo.text)
            for token_adjetivo in token_adyacente.rights:
                if token_adjetivo.dep_ == 'ADJ':
                    palabras.append(token_adjetivo.text)
    return palabras

def n_gramas(oracion, palabra, numero):
    """
    Extrae n-gramas (secuencias de n palabras) alrededor de una palabra clave en una oración.
    """
    palabras = limpiar_oracion(oracion).split() # Limpia la oración y la divide en palabras 
    index_palabra = [] # Inicializa una lista para guardar el índice de la palabra clave en la lista de palabras

    for index, word in enumerate(palabras): # Itera sobre las palabras para encontrar la palabra clave
        if word.lower() == palabra.lower() or word.lower() == stemmer.stem(palabra.lower()): # Busca la palabra clave o su raíz
            index_palabra = index
            break

    palabras_anteriores = palabras[max(0, index_palabra - numero):index_palabra] # Extrae las palabras anteriores a la palabra clave
    palabras_posteriores = palabras[index_palabra + 1:index_palabra + 1 + numero] # Extrae las palabras posteriores a la palabra clave

    return palabras_anteriores + [palabra] + palabras_posteriores # Devuelve una lista con las palabras anteriores, la palabra clave y las palabras posteriores


def extraccion_contexto(oracion, palabra_relevante):
    """
    Extrae el contexto de una palabra clave en una oración, incluyendo modificadores.
    """
    doc = nlp(oracion) # Analiza la oración con spaCy
    token_palabra_relevante = None # Inicializa la variable para guardar el token de la palabra clave
    for token in doc: # Busca el token de la palabra clave en la oración analizada
        if token.text.lower() == palabra_relevante:
            token_palabra_relevante = token
            break

    if not token_palabra_relevante: # Si no se encuentra la palabra clave
        return "Palabra relevante no encontrada en la oración"

    modificadores = [] # Inicializa una lista para guardar los modificadores de la palabra clave
    for token in doc: # Itera sobre los tokens de la oración analizada
        # Busca modificadores de la palabra clave (adjetivos, adverbios, etc.)
        if token.head == token_palabra_relevante and token.dep_ in ["amod", "advmod", "nsubj", "nummod", "det"]: 
            modificadores.append(token.text)

    return modificadores # Devuelve la lista de modificadores


def limpiar_oracion(oracion):

    # Usar una expresión regular para encontrar y reemplazar todos los caracteres que no sean letras
    oracion_limpia = re.sub(r'[^a-zA-Z\sáéíóúüÁÉÍÓÚÜñÑ]', ' ', oracion) 
    # Devolver la oración limpia
    return oracion_limpia

def palabras_sin_repetir(lista_de_listas):
    palabras = set()  # Utilizamos un conjunto para evitar palabras duplicadas  
    for sublista in lista_de_listas:
        for palabra in sublista:
            palabras.add(palabra) 

    return list(palabras)  # Convertimos el conjunto de palabras de nuevo a una lista y la devolvemos

def combinacion(oracion, lista_palabras):
    """
    Extrae las palabras de una oración que están presentes en una lista dada, 
    evitando duplicados y manejando algunas variaciones simples.

    Args:
        oracion: La oración de entrada como una cadena de texto.
        lista_palabras: Una lista de palabras a buscar en la oración.

    Returns:
        Una lista de palabras de la oración que se encontraron en la lista de palabras, sin duplicados.
    """
    resultado = [] # Inicializa una lista vacía para almacenar las palabras encontradas.

    for token in oracion.split(): # Itera sobre cada palabra (token) en la oración.
        if (token in lista_palabras) or (token[0:-1] in lista_palabras): # Verifica si el token o su forma sin el último carácter está en la lista de palabras.
            if not token in resultado: # Verifica si el token ya está en la lista de resultados para evitar duplicados.
                resultado.append(token) # Agrega el token a la lista de resultados.

    return resultado # Devuelve la lista de palabras encontradas.

def convertir_lista_oracion(lista_palabras):

    oracion = " ".join(lista_palabras)  # Unir las palabras con espacios
    return oracion



#///////////////////////////////////////////////////////////////////////////////////////////////////////



def process_xml_to_csv(xml_filename, csv_filename):
    """
    Procesa un archivo XML de reseñas y crea un archivo CSV, extrayendo solo la primera palabra del aspecto.
    """
    tree = ET.parse(xml_filename)
    root = tree.getroot()

    data = []
    for review in root.findall('Review'):
        for sentence in review.findall('sentences/sentence'):
            text = sentence.find('text').text
            for opinion in sentence.findall('Opinions/Opinion'):
                target = opinion.get('target')
                polarity = opinion.get('polarity')

                if target is not None:
                    aspect = target.split()[0] # Extrae la primera palabra
                else:
                    aspect = "GENERAL" # Para opiniones sin target específico

                if polarity == 'positive':
                    pol = 1
                elif polarity == "neutral":
                    pol = 2
                else:
                    pol = 0

                data.append([text, aspect, pol])

    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, delimiter=';')
        writer.writerow(['texto', 'aspecto', 'polaridad'])
        writer.writerows(data)





In [41]:
# Ejemplo de uso: Reemplaza 'input.xml' y 'output.csv' con los nombres de tus archivos.
xml_file = '../Data/SemEval/SemEval-2016ABSA Restaurants-Spanish_Test.xml'
csv_file = 'output.csv'
process_xml_to_csv(xml_file, csv_file)
print(f"Archivo CSV creado: {csv_file}")


Archivo CSV creado: output.csv


### Añadir columna segmento

In [44]:
import pandas as pd

def procesar_csv(ruta_archivo):
    """
    Recorre un archivo CSV y imprime el texto y el aspecto de cada fila.

    Args:
        ruta_archivo: La ruta al archivo CSV.
    """

    df = pd.read_csv(ruta_archivo, sep=";")
    df["Segmento"] = ""
    lista_segmentos = []
    try:
        for index, fila in df.iterrows():
            texto = fila['review']
            aspecto = fila['aspect']
            if aspecto == "vacio":
                segmento = texto
            else:
                segmento = extraer_segmento(texto,aspecto)
                
            lista_segmentos.append(segmento)

    except FileNotFoundError:
        print(f"Error: Archivo '{ruta_archivo}' no encontrado.")
    except Exception as e:
        print(f"Error al procesar el archivo: {e}")

    df["Segmento"] = lista_segmentos
    df.to_csv(ruta_archivo, index=False, sep=";")


# Reemplaza 'tu_archivo.csv' con la ruta real a tu archivo CSV
ruta_archivo = '../Segment_extraction/nuevo_archivo.csv'  
procesar_csv(ruta_archivo)



### Eliminar Filas con Polaridad Neutra

In [35]:
import pandas as pd

# Lee el archivo CSV
df = pd.read_csv("../Segment_extraction/nuevo_archivo.csv", sep=";") 

# Filtra las filas donde la columna 'polaridad' no es igual a 2
df_filtrado = df[df["polarity"] != 'neutral']

# Guarda el DataFrame filtrado en un nuevo archivo CSV
df_filtrado.to_csv("archivo_filtrado.csv", sep=";", index=False)

print("Archivo filtrado creado correctamente.")


Archivo filtrado creado correctamente.
